In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd

# Training 데이터 파일 목록
train_files = [
    '/content/drive/MyDrive/KOBART/data/train/시사교육 (2).csv',
    '/content/drive/MyDrive/KOBART/data/train/식음료 (2).csv',
    '/content/drive/MyDrive/KOBART/data/train/여가생활 (2).csv',
    '/content/drive/MyDrive/KOBART/data/train/의료 (2).csv',
    '/content/drive/MyDrive/KOBART/data/train/교육 (2).csv',

]

# Test 파일 경로
test_file = "/content/drive/MyDrive/KOBART/data/validation/dev_v1 (5).csv"

# Training 데이터 로드 및 병합
train_data = pd.concat([pd.read_csv(file) for file in train_files], ignore_index=True)

# Test 데이터 로드
test_data = pd.read_csv(test_file)

# 데이터 확인
print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

# 샘플 확인
print("Training data sample:")
print(train_data.head())

print("\nTest data sample:")
print(test_data.head())


Training data shape: (254520, 3)
Test data shape: (35000, 3)
Training data sample:
   Unnamed: 0                                               text  \
0           0  북구청 때문에 너희 집 앞이 너무 무서워 북구청은 의심환자만 은밀하게가 코로나 동선...   
1           1  #@이름# 어제 그 동묘 고양이 사건전말 떳다 ㅋㅋㅋ 헐 모양 근데 사진은 잇엇자나...   
2           2  한국사 공부하면 화가 많아지는건가...ㅋㅋㅋㅋㅋㅋㅋㅋ 맞아... 근데 나중에 한국사...   
3           3  합격예상?? 어땟어 저번보다는 잘봣어!!! ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 저번보다는 확실히 자신...   
4           4  음 중강고사 뭐 죽쒔죠 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 9그럼 출석열심히하는거 의미없어요#...   

                                         abstractive  
0  코로나 의심 환자들이 북구청으로 들어가는 길은 따로 있어서 집 앞으로 오는 것을 걱...  
1       어제 그 동묘 고양이 사건의 전말이 떴는데 시시티브이(cctv) 영상도 나왔다.  
2              한국사 공부를 하면 친일파 청산을 하지 않은 것에 화가 나게 된다.  
3                         저번보다 잘 보기는 했는데 합격은 잘 모르겠다.  
4  중간고사를 망해서 출석 열심히 해봤자 의미가 없다며 장학금은 기대도 안 한다고 얘기했다.  

Test data sample:
   Unnamed: 0                                               text  \
0           0  ㅠㅠ퇴근했옹~? 응ㅋㅋㅋㅋ퇴근했지ㅜ 나두30분정도뒤에 퇴근할꺼같앙ㅠ 오♡♡♡♡♡ 우...   
1           1  아주 고와ㅋㅋㅋ

In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")

# Dataset 클래스 정의
class SummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_len=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]

        text_enc = self.tokenizer(
            text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )
        summary_enc = self.tokenizer(
            summary, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "input_ids": text_enc["input_ids"].squeeze(),
            "attention_mask": text_enc["attention_mask"].squeeze(),
            "labels": summary_enc["input_ids"].squeeze(),
        }


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [ ]:
# Training 데이터셋
train_dataset = SummaryDataset(
    texts=train_data["text"].tolist(),
    summaries=train_data["abstractive"].tolist(),
    tokenizer=tokenizer
)

# Test 데이터셋
test_dataset = SummaryDataset(
    texts=test_data["text"].tolist(),
    summaries=test_data["abstractive"].tolist(),
    tokenizer=tokenizer
)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)


In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optimizer 정의
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 학습 루프
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.048926014453172684
Epoch 2, Loss: 0.07381361722946167
Epoch 3, Loss: 0.056066010147333145


In [ ]:
# 모델 평가
model.eval()
test_results = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=50,
            num_beams=5,
            early_stopping=True
        )

        summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
        test_results.extend(summaries)

# 결과를 원래 데이터프레임에 저장
test_data["KOBART_abstractive"] = test_results


In [ ]:
# 결과를 CSV 파일로 저장
output_file = "/content/drive/MyDrive/KOBART/data/validation/test_with_kobart_summaries.csv"
test_data.to_csv(output_file, index=False)

print(f"Updated test data saved to {output_file}")

Updated test data saved to /content/drive/MyDrive/KOBART/data/validation/test_with_kobart_summaries.csv
